In [ ]:
# Install noveum-trace from PyPI and testing dependencies
%pip install noveum-trace python-dotenv openai anthropic
%pip install --upgrade noveum-trace pip


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
# Note: Using robust path detection since __file__ is not available in Jupyter notebooks
try:
    # Try to get current working directory
    current_dir = os.getcwd()
    print(f"📁 Current directory: {current_dir}")
    
    # Look for .env file in current directory and parent directories
    env_file_found = False
    search_dir = current_dir
    
    for _ in range(5):  # Search up to 5 levels up
        env_file = os.path.join(search_dir, '.env')
        if os.path.exists(env_file):
            print(f"📄 Found .env file: {env_file}")
            load_dotenv(env_file)
            env_file_found = True
            break
        search_dir = os.path.dirname(search_dir)
        if search_dir == os.path.dirname(search_dir):  # Reached root
            break
    
    if not env_file_found:
        print("ℹ️  No .env file found - continuing without it")
        
except Exception as e:
    print(f"⚠️  Error loading .env file: {e}")
    print("ℹ️  Continuing without .env file")

# Set up environment variables for testing
# Replace with your actual API key or set in .env file
if not os.getenv('NOVEUM_API_KEY'):
    # For testing purposes, you can set a dummy API key
    # In production, use your actual Noveum API key
    os.environ['NOVEUM_API_KEY'] = 'noveum_API_KEY'
    print("⚠️  Using dummy API key for testing. Set NOVEUM_API_KEY environment variable for production use.")
else:
    print("✅ NOVEUM_API_KEY found i")

# Optional: Set OpenAI API key for LLM examples
if not os.getenv('OPENAI_API_KEY'):
    print("ℹ️  OPENAI_API_KEY not found. LLM examples will use mock responses.")
else:
    print("✅ OPENAI_API_KEY found in environment")

print("\n📋 Environment Variables Status:")
print(f"NOVEUM_API_KEY: {'✓' if os.getenv('NOVEUM_API_KEY') else '✗'}")
print(f"OPENAI_API_KEY: {'✓' if os.getenv('OPENAI_API_KEY') else '✗'}")


## 🔄 FLUSH HELPER: Automatic Trace Sending

To ensure all traces are sent immediately to your endpoint, we'll create a helper function that can be called after any traced operation.


In [ ]:
# 🔄 FLUSH HELPER FUNCTIONS FOR IMMEDIATE TRACE SENDING

def flush_traces(operation_name="Operation"):
    """
    Helper function to flush traces immediately to endpoint.
    Call this after any traced operation to ensure traces are sent right away.
    """
    try:
        noveum_trace.flush()
        print(f"📤 ✅ {operation_name} traces flushed successfully")
    except Exception as e:
        print(f"📤 ⚠️  {operation_name} flush warning: {e}")

def auto_flush_decorator(func):
    """
    Decorator that automatically flushes traces after function execution.
    Use this for any function that contains traced operations.
    """
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        flush_traces(func.__name__)
        return result
    return wrapper

# Test the flush helper
print("🔧 Flush helper functions initialized")
print("📋 Usage:")
print("  - Call flush_traces('operation_name') after any traced operation")
print("  - Use @auto_flush_decorator on functions containing traced operations")
print("  - This ensures immediate trace sending to your endpoint")


In [ ]:
import noveum_trace
from noveum_trace import trace, trace_agent, trace_llm, trace_tool
import logging

# Enable detailed logging to debug transport issues
logging.basicConfig(level=logging.DEBUG)
transport_logger = logging.getLogger('noveum_trace.transport')
transport_logger.setLevel(logging.DEBUG)
ENDPOINT = "https://api.noveum.ai/api"

# Initialize the SDK with proper endpoint configuration
try:
    # IMPORTANT: The SDK will append "/v1/traces" to your endpoint
    # So "https://noveum-trace.free.beeceptor.com" becomes "https://noveum-trace.free.beeceptor.com/v1/traces"
    noveum_trace.init(
        api_key=os.getenv('NOVEUM_API_KEY'),
        project="jupyter-test-project", 
        environment="development",
        endpoint=ENDPOINT,  # SDK will add /v1/traces automatically
        debug=True,  # Enable debug mode for testing
        
        # Transport configuration for better reliability
        transport_config={
            "timeout": 10,           # 10 second timeout
            "retry_attempts": 2,     # Retry failed requests 2 times
            "batch_size": 10,        # Smaller batches for demo
            "batch_timeout": 2.0,    # Send batches every 2 seconds
            "compression": False     # Disable compression for debugging
        },
        
        # Tracing configuration
        tracing_config={
            "sample_rate": 1.0,      # Trace 100% of operations
            "capture_errors": True,
            "capture_stack_traces": True
        }
    )
    
    print("✅ Noveum Trace SDK initialized successfully!")
    print("📊 Project: jupyter-test-project")
    print("🔧 Environment: development") 
    print(f"🌐 Endpoint: {ENDPOINT}/v1/traces (auto-appended)")
    print("🔍 Debug logging enabled - check console for HTTP request details")
    
    # Get the current configuration to verify settings
    config = noveum_trace.get_config()
    print(f"📋 Config verified - Endpoint: {config.transport.endpoint}")
    print(f"📦 Batch size: {config.transport.batch_size}")
    print(f"⏱️  Batch timeout: {config.transport.batch_timeout}s")
    
except Exception as e:
    print(f"❌ Error initializing SDK: {e}")
    print("Continuing with demo - traces will be logged locally")
    import traceback
    traceback.print_exc()


In [ ]:
# 🔄 FLUSH AFTER LLM CALL TRACING
# This ensures the @trace_llm decorator traces are sent immediately

flush_traces("LLM Call Tracing (call_language_model)")


In [ ]:
# Mock LLM responses for testing (replace with actual API calls if you have keys)
import time
import random
import os
from noveum_trace import trace_llm

def mock_openai_call(prompt: str, model: str = "gpt-4") -> str:
    """Mock OpenAI API call for testing."""
    responses = [
        "This is a mock response from the language model.",
        "Here's a simulated AI response for testing purposes.",
        "Mock LLM output to demonstrate tracing functionality."
    ]
    time.sleep(0.3)  # Simulate API call latency
    return random.choice(responses)

@trace_llm
def call_language_model(prompt: str, model: str = "gpt-4") -> str:
    """Call a language model with tracing."""
    print(f"🤖 Calling {model} with prompt: {prompt[:50]}...")

    # Use real OpenAI API if available, otherwise use mock
    if os.getenv('OPENAI_API_KEY'):
        try:
            import openai
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"⚠️  OpenAI API call failed: {e}. Using mock response.")
            return mock_openai_call(prompt, model)
    else:
        print("📝 Using mock LLM response (no API key provided)")
        return mock_openai_call(prompt, model)

# Test LLM tracing
prompt = "Explain the benefits of observability in AI systems."
response = call_language_model(prompt)
print(f"\n🎯 LLM Response: {response}")


In [ ]:
# 🔄 FLUSH AFTER ENHANCED LLM TRACING
# This ensures all enhanced LLM traces (Anthropic, OpenAI, Google) are sent immediately

flush_traces("Enhanced LLM Tracing (Anthropic + OpenAI + Google)")


In [ ]:
# 🔄 FLUSH AFTER RETRIEVAL SYSTEM TRACING  
# This ensures all @trace_retrieval decorator traces are sent immediately

flush_traces("Retrieval System Tracing (Vector + Keyword + Hybrid)")


In [ ]:
# Enhanced LLM Tracing Examples

# Test different LLM providers with comprehensive metadata
@trace_llm(provider="anthropic", capture_tokens=True, estimate_costs=True)
def call_anthropic(prompt: str, model: str = "claude-3-haiku") -> str:
    """Call Anthropic Claude with tracing."""
    print(f"🧠 Calling {model} with prompt: {prompt[:50]}...")
    
    # Mock Anthropic response
    time.sleep(0.4)
    responses = [
        "Observability in AI systems provides critical insights into model behavior and performance.",
        "Tracing AI workflows enables debugging, optimization, and compliance monitoring.",
        "Comprehensive monitoring helps ensure AI system reliability and user trust."
    ]
    return random.choice(responses)

# Test with custom metadata and tags
@trace_llm(
    provider="openai", 
    capture_prompts=True, 
    capture_completions=True,
    metadata={"experiment": "demo", "version": "1.0"},
    tags={"environment": "notebook", "user": "demo"}
)
def call_llm_with_metadata(prompt: str, model: str = "gpt-4") -> str:
    """LLM call with custom metadata and tags."""
    print(f"📋 Enhanced LLM call with metadata: {prompt[:40]}...")
    time.sleep(0.3)
    return f"Enhanced response for: {prompt[:20]}..."

# Test Google AI provider
@trace_llm(provider="google", capture_tokens=True, redact_pii=True)
def call_google_ai(prompt: str, model: str = "gemini-pro") -> str:
    """Call Google AI with PII redaction."""
    print(f"🟢 Calling {model} with PII protection: {prompt[:40]}...")
    time.sleep(0.5)
    return "Google AI response with PII redaction enabled for sensitive data handling."

# Test various LLM providers
print("🤖 Testing Enhanced LLM Tracing...")

anthropic_response = call_anthropic("What are the key benefits of AI observability?")
print(f"\n🧠 Anthropic Response: {anthropic_response}")

metadata_response = call_llm_with_metadata("Summarize the importance of AI monitoring")
print(f"\n📋 Enhanced Response: {metadata_response}")

google_response = call_google_ai("How does tracing help with AI compliance?")
print(f"\n🟢 Google AI Response: {google_response}")

print("\n✅ Enhanced LLM tracing completed!")


In [ ]:
# 🔄 FLUSH AFTER ENHANCED MULTI-AGENT SYSTEM TRACING
# This ensures all @trace_agent decorator traces are sent immediately

flush_traces("Enhanced Multi-Agent System (Data Analyst + Content Curator + Synthesis + Orchestrator)")


In [ ]:
# 🔄 FLUSH AFTER CONTEXT MANAGERS AND STREAMING
# This ensures all context manager traces are sent immediately

flush_traces("Context Managers and Streaming (trace_llm_call + trace_agent_operation + trace_operation + streaming)")


In [ ]:
# Import the retrieval decorator
from noveum_trace import trace_retrieval
from typing import Dict, Any, Optional
import time

# Vector search with comprehensive tracing
@trace_retrieval(
    retrieval_type="vector_search",
    index_name="knowledge_base",
    capture_query=True,
    capture_results=True,
    capture_scores=True,
    metadata={"index_version": "v2.1", "embedding_model": "text-embedding-ada-002"}
)
def vector_search(query: str, top_k: int = 5) -> Dict[str, Any]:
    """Perform vector search with tracing."""
    print(f"🔍 Vector Search: Finding top {top_k} results for '{query}'")
    
    # Simulate vector search
    time.sleep(0.3)
    
    # Mock search results with scores
    results = []
    for i in range(top_k):
        results.append({
            "document_id": f"doc_{i+1}",
            "content": f"Relevant content for '{query}' - document {i+1}",
            "score": 0.95 - (i * 0.1),
            "metadata": {"source": f"source_{i+1}.pdf", "page": i+1}
        })
    
    search_result = {
        "query": query,
        "total_results": top_k,
        "results": results,
        "search_time_ms": 300,
        "index_stats": {"total_docs": 10000, "dimensions": 1536}
    }
    
    print(f"✅ Found {len(results)} relevant documents")
    return search_result

# Keyword search with metadata capture
@trace_retrieval(
    retrieval_type="keyword_search",
    index_name="text_corpus",
    capture_metadata=True,
    tags={"search_type": "fulltext", "language": "en"}
)
def keyword_search(query: str, filters: Optional[Dict] = None) -> Dict[str, Any]:
    """Perform keyword search with filtering."""
    print(f"🔎 Keyword Search: '{query}' with filters: {filters}")
    
    time.sleep(0.2)
    
    # Mock keyword search results
    results = [
        {"doc_id": "kw_1", "title": "AI Observability Guide", "snippet": "...observability in AI..."},
        {"doc_id": "kw_2", "title": "Tracing Best Practices", "snippet": "...tracing methodologies..."},
        {"doc_id": "kw_3", "title": "Monitoring AI Systems", "snippet": "...monitoring strategies..."}
    ]
    
    return {
        "query": query,
        "filters": filters or {},
        "results": results,
        "total_matches": len(results)
    }

# Test retrieval operations
print("🔍 Testing Retrieval System Tracing...")

# Test vector search
vector_result = vector_search("benefits of AI observability", top_k=3)
print(f"\n🔍 Vector Search Results: {len(vector_result['results'])} documents")

# Test keyword search with filters
keyword_result = keyword_search("AI monitoring", filters={"category": "technical", "year": 2024})
print(f"\n🔎 Keyword Search Results: {keyword_result['total_matches']} matches")

print("\n✅ Retrieval tracing completed!")


In [ ]:
# Enhanced Multi-Agent System Examples
# Import required types to ensure they're available in this cell
from typing import Dict, Any, List

# Specialized agents with different roles and capabilities
@trace_agent(
    agent_id="data_analyst",
    role="analyst",
    agent_type="specialist",
    capabilities=["data_analysis", "statistical_modeling", "visualization"],
    capture_reasoning=True,
    metadata={"specialization": "quantitative_analysis", "confidence_threshold": 0.8}
)
def data_analyst_agent(data: Dict[str, Any]) -> Dict[str, Any]:
    """Specialized data analysis agent."""
    print(f"📊 Data Analyst: Analyzing dataset with {len(data.get('samples', []))} samples")
    
    time.sleep(0.5)
    
    # Mock data analysis
    analysis = {
        "agent_id": "data_analyst",
        "analysis_type": "quantitative",
        "findings": {
            "data_quality": 0.92,
            "pattern_confidence": 0.87,
            "anomalies_detected": 3,
            "recommendations": [
                "Data quality is high with 92% confidence",
                "3 anomalies detected requiring investigation",
                "Statistical patterns show strong correlation"
            ]
        },
        "reasoning_steps": [
            "Loaded and validated input data",
            "Applied statistical analysis methods", 
            "Identified patterns and anomalies",
            "Generated confidence-based recommendations"
        ],
        "processing_time": 0.5
    }
    
    print(f"✅ Analysis complete: {analysis['findings']['data_quality']:.2f} quality score")
    return analysis

@trace_agent(
    agent_id="content_curator",
    role="curator",
    agent_type="content_specialist", 
    capabilities=["content_filtering", "quality_assessment", "summarization"],
    capture_tools=True
)
def content_curator_agent(content_list: List[Dict]) -> Dict[str, Any]:
    """Content curation and quality assessment agent."""
    print(f"📝 Content Curator: Processing {len(content_list)} content items")
    
    time.sleep(0.4)
    
    # Mock content curation using tools
    high_quality_content = []
    for i, content in enumerate(content_list):
        if i < 3:  # Mock: keep first 3 as high quality
            high_quality_content.append({
                **content,
                "quality_score": 0.9 - (i * 0.05),
                "curation_reason": "Meets quality standards"
            })
    
    curation_result = {
        "agent_id": "content_curator",
        "input_count": len(content_list),
        "curated_count": len(high_quality_content),
        "curated_content": high_quality_content,
        "tools_used": ["quality_scorer", "content_filter", "summarizer"],
        "curation_metrics": {
            "retention_rate": len(high_quality_content) / len(content_list),
            "average_quality": sum(item["quality_score"] for item in high_quality_content) / len(high_quality_content)
        }
    }
    
    print(f"✅ Curated {len(high_quality_content)}/{len(content_list)} items")
    return curation_result

# Test the enhanced multi-agent system
print("🤖 Testing Enhanced Multi-Agent System...")

# Mock data for demonstration
sample_data = {"samples": [f"sample_{i}" for i in range(100)]}
sample_content = [
    {"id": 1, "title": "AI Observability", "content": "Content about observability"},
    {"id": 2, "title": "Tracing Systems", "content": "Content about tracing"},
    {"id": 3, "title": "Monitoring Tools", "content": "Content about monitoring"},
    {"id": 4, "title": "Low Quality", "content": "Poor content"}
]

# Test individual agents
analyst_result = data_analyst_agent(sample_data)
print(f"\n📊 Data Analysis: {analyst_result['findings']['data_quality']:.2f} quality score")

curator_result = content_curator_agent(sample_content)
print(f"\n📝 Content Curation: {curator_result['curated_count']}/{curator_result['input_count']} items retained")

print("\n✅ Enhanced multi-agent system testing completed!")


In [ ]:
# Import context managers and streaming features
from noveum_trace import (
    trace_llm_call, trace_agent_operation, trace_operation, 
    streaming_llm, trace_streaming, ThreadContext
)
from typing import Dict, Any, Iterator

# Context Manager Examples - Inline Tracing

def process_user_query_with_context_managers(user_input: str) -> str:
    """Demonstrate inline tracing with context managers."""
    print(f"🔄 Processing user query: '{user_input[:40]}...'")
    
    # Some preprocessing (not traced)
    cleaned_input = user_input.strip().lower()
    
    # Trace just the LLM call using context manager
    with trace_llm_call(model="gpt-4", provider="openai", operation="query_processing") as span:
        print("🤖 Making LLM call within context manager...")
        time.sleep(0.4)
        
        # Mock LLM response
        response = f"Processed response for: {cleaned_input[:30]}..."
        
        # Add custom attributes to the span
        span.set_attributes({
            "llm.input_length": len(cleaned_input),
            "llm.output_length": len(response),
            "llm.processing_type": "query_understanding"
        })
        
        print(f"✅ LLM response generated: {len(response)} characters")
    
    # Post-processing (not traced)
    final_response = f"Final: {response}"
    print(f"📤 Final response: {final_response[:50]}...")
    
    return final_response

# Agent operation context manager
def agent_task_with_context_manager(task: str) -> Dict[str, Any]:
    """Demonstrate agent operation tracing with context manager."""
    print(f"🤖 Agent Task: '{task}'")
    
    with trace_agent_operation(
        agent_type="task_agent", 
        operation="task_execution",
        capabilities=["task_planning", "execution", "monitoring"]
    ) as span:
        print("⚙️  Executing agent task...")
        time.sleep(0.3)
        
        # Mock agent work
        result = {
            "task": task,
            "status": "completed",
            "steps_executed": 5,
            "success_rate": 0.95
        }
        
        # Add agent-specific attributes
        span.set_attributes({
            "agent.task_complexity": "medium",
            "agent.steps_executed": result["steps_executed"],
            "agent.success_rate": result["success_rate"]
        })
        
        print(f"✅ Agent task completed with {result['success_rate']:.1%} success rate")
    
    return result

# Generic operation context manager (FIXED SYNTAX)
def complex_operation_with_tracing() -> Dict[str, Any]:
    """Demonstrate generic operation tracing."""
    print("🔧 Starting complex operation...")
    
    # CORRECT SYNTAX: trace_operation(operation_name, attributes=...)
    with trace_operation("complex_data_processing", 
                        attributes={"operation_type": "data_pipeline", "complexity": "high"}) as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "complex_data_processing",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

# Test context managers
print("🔄 Testing Context Managers...")

print("\\n1️⃣ Context Manager Examples:")
query_result = process_user_query_with_context_managers("What are the benefits of AI observability?")

agent_result = agent_task_with_context_manager("Analyze system performance metrics")

operation_result = complex_operation_with_tracing()

print("\\n✅ Context managers testing completed!")


In [ ]:
# Import advanced features
from noveum_trace import (
    auto_instrument, get_instrumented_libraries, is_instrumented,
    create_traced_openai_client, create_traced_agent, TracedOpenAIClient,
    start_trace, start_span, get_current_trace, get_current_span
)

# Auto-Instrumentation Examples

def test_auto_instrumentation():
    """Test automatic instrumentation of libraries."""
    print("🔧 Testing Auto-Instrumentation...")
    
    # Check available instrumentations
    try:
        available = noveum_trace.get_available_instrumentations()
        print(f"📦 Available instrumentations: {available}")
    except Exception as e:
        print(f"ℹ️  Available instrumentations: {e}")
    
    # Enable auto-instrumentation for OpenAI (if not already enabled)
    try:
        if not is_instrumented("openai"):
            print("🔌 Enabling OpenAI auto-instrumentation...")
            auto_instrument("openai")
            print("✅ OpenAI auto-instrumentation enabled")
        else:
            print("✅ OpenAI already instrumented")
    except Exception as e:
        print(f"ℹ️  Auto-instrumentation: {e}")
    
    # Check instrumented libraries
    try:
        instrumented = get_instrumented_libraries()
        print(f"🔍 Currently instrumented: {instrumented}")
    except Exception as e:
        print(f"ℹ️  Instrumented libraries: {e}")
    
    return []

# Proxy Objects for Enhanced Control

def test_traced_openai_client():
    """Test traced OpenAI client proxy."""
    print("🔄 Testing Traced OpenAI Client...")
    
    # Create traced OpenAI client (even without real API key)
    try:
        traced_client = create_traced_openai_client(
            api_key="mock-key-for-demo",
            trace_completions=True,
            trace_embeddings=True,
            capture_content=True
        )
        print("✅ Traced OpenAI client created")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        # In real usage: response = traced_client.chat.completions.create(...)
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")

def test_traced_agent_proxy():
    """Test traced agent proxy for enhanced agent monitoring."""
    print("🤖 Testing Traced Agent Proxy...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # Create traced agent proxy with CORRECT PARAMETERS
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        agent_type="traced_demo_agent",  # CORRECT: agent_type instead of agent_id
        capabilities=["thinking", "acting", "planning"],
        trace_config={"capture_inputs": True, "capture_outputs": True}
    )
    
    print("✅ Traced agent proxy created")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")

# Manual Span Creation and Management

def test_manual_tracing():
    """Test manual trace and span creation."""
    print("🔍 Testing Manual Tracing...")
    
    try:
        # Start a manual trace
        trace = start_trace("manual_demo_trace")
        print(f"✅ Started trace: {trace.trace_id}")
        
        # Create nested spans manually
        with trace.span("parent_operation") as parent_span:
            parent_span.set_attributes({
                "operation.type": "parent",
                "operation.importance": "high"
            })
            print("📊 Parent span created")
            
            # Child span 1
            with parent_span.create_child_span("child_operation_1") as child1:
                child1.set_attributes({
                    "operation.type": "child",
                    "child.number": 1
                })
                time.sleep(0.2)
                print("🔹 Child span 1 completed")
            
            # Child span 2  
            with parent_span.create_child_span("child_operation_2") as child2:
                child2.set_attributes({
                    "operation.type": "child",
                    "child.number": 2,
                    "child.data_processed": 500
                })
                time.sleep(0.3)
                print("🔹 Child span 2 completed")
            
            print("📊 Parent operation completed")
        
        # Finish trace
        trace.finish()
        print(f"✅ Manual trace completed: {trace.trace_id}")
        
    except Exception as e:
        print(f"ℹ️  Manual tracing demo: {e}")

# Error Handling and Edge Cases

def test_error_handling():
    """Test error handling and edge cases."""
    print("⚠️  Testing Error Handling...")
    
    # Test error capture in traced function
    @noveum_trace.trace(capture_errors=True, capture_stack_trace=True)
    def operation_with_error(should_fail: bool = False):
        if should_fail:
            raise ValueError("This is a demo error for testing")
        return "Success!"
    
    # Test successful operation
    try:
        result = operation_with_error(should_fail=False)
        print(f"✅ Successful operation: {result}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    
    # Test error capture
    try:
        result = operation_with_error(should_fail=True)
        print(f"Unexpected success: {result}")
    except ValueError as e:
        print(f"✅ Error captured successfully: {e}")

# Run all advanced feature tests
print("🚀 Testing Auto-Instrumentation and Advanced Features...")

print("\\n1️⃣ Auto-Instrumentation:")
instrumented_libs = test_auto_instrumentation()

print("\\n2️⃣ Proxy Objects:")
test_traced_openai_client()
test_traced_agent_proxy()

print("\\n3️⃣ Manual Tracing:")
test_manual_tracing()

print("\\n4️⃣ Error Handling:")
test_error_handling()

print("\\n✅ Advanced features testing completed!")


In [ ]:
# Enhanced LLM Tracing Examples

# Test different LLM providers with comprehensive metadata
@trace_llm(provider="anthropic", capture_tokens=True, estimate_costs=True)
def call_anthropic(prompt: str, model: str = "claude-3-haiku") -> str:
    """Call Anthropic Claude with tracing."""
    print(f"🧠 Calling {model} with prompt: {prompt[:50]}...")
    
    # Mock Anthropic response
    time.sleep(0.4)
    responses = [
        "Observability in AI systems provides critical insights into model behavior and performance.",
        "Tracing AI workflows enables debugging, optimization, and compliance monitoring.",
        "Comprehensive monitoring helps ensure AI system reliability and user trust."
    ]
    return random.choice(responses)

# Test with custom metadata and tags
@trace_llm(
    provider="openai", 
    capture_prompts=True, 
    capture_completions=True,
    metadata={"experiment": "demo", "version": "1.0"},
    tags={"environment": "notebook", "user": "demo"}
)
def call_llm_with_metadata(prompt: str, model: str = "gpt-4") -> str:
    """LLM call with custom metadata and tags."""
    print(f"📋 Enhanced LLM call with metadata: {prompt[:40]}...")
    time.sleep(0.3)
    return f"Enhanced response for: {prompt[:20]}..."

# Test Google AI provider
@trace_llm(provider="google", capture_tokens=True, redact_pii=True)
def call_google_ai(prompt: str, model: str = "gemini-pro") -> str:
    """Call Google AI with PII redaction."""
    print(f"🟢 Calling {model} with PII protection: {prompt[:40]}...")
    time.sleep(0.5)
    return "Google AI response with PII redaction enabled for sensitive data handling."

# Test various LLM providers
print("🤖 Testing Enhanced LLM Tracing...")

anthropic_response = call_anthropic("What are the key benefits of AI observability?")
print(f"\n🧠 Anthropic Response: {anthropic_response}")

metadata_response = call_llm_with_metadata("Summarize the importance of AI monitoring")
print(f"\n📋 Enhanced Response: {metadata_response}")

google_response = call_google_ai("How does tracing help with AI compliance?")
print(f"\n🟢 Google AI Response: {google_response}")

print("\n✅ Enhanced LLM tracing completed!")


In [ ]:
# 🔄 FLUSH AFTER CORRECTED TRACE_OPERATION EXAMPLES
# This ensures all corrected context manager traces are sent immediately

flush_traces("Corrected trace_operation Examples (Fixed Syntax)")


In [ ]:
# Import the retrieval decorator and missing typing imports
from noveum_trace import trace_retrieval
from typing import Dict, Any, Optional, List, Iterator

# Vector search with comprehensive tracing
@trace_retrieval(
    retrieval_type="vector_search",
    index_name="knowledge_base",
    capture_query=True,
    capture_results=True,
    capture_scores=True,
    metadata={"index_version": "v2.1", "embedding_model": "text-embedding-ada-002"}
)
def vector_search(query: str, top_k: int = 5) -> Dict[str, Any]:
    """Perform vector search with tracing."""
    print(f"🔍 Vector Search: Finding top {top_k} results for '{query}'")
    
    # Simulate vector search
    time.sleep(0.3)
    
    # Mock search results with scores
    results = []
    for i in range(top_k):
        results.append({
            "document_id": f"doc_{i+1}",
            "content": f"Relevant content for '{query}' - document {i+1}",
            "score": 0.95 - (i * 0.1),
            "metadata": {"source": f"source_{i+1}.pdf", "page": i+1}
        })
    
    search_result = {
        "query": query,
        "total_results": top_k,
        "results": results,
        "search_time_ms": 300,
        "index_stats": {"total_docs": 10000, "dimensions": 1536}
    }
    
    print(f"✅ Found {len(results)} relevant documents")
    return search_result

# Keyword search with metadata capture
@trace_retrieval(
    retrieval_type="keyword_search",
    index_name="text_corpus",
    capture_metadata=True,
    tags={"search_type": "fulltext", "language": "en"}
)
def keyword_search(query: str, filters: Optional[Dict] = None) -> Dict[str, Any]:
    """Perform keyword search with filtering."""
    print(f"🔎 Keyword Search: '{query}' with filters: {filters}")
    
    time.sleep(0.2)
    
    # Mock keyword search results
    results = [
        {"doc_id": "kw_1", "title": "AI Observability Guide", "snippet": "...observability in AI..."},
        {"doc_id": "kw_2", "title": "Tracing Best Practices", "snippet": "...tracing methodologies..."},
        {"doc_id": "kw_3", "title": "Monitoring AI Systems", "snippet": "...monitoring strategies..."}
    ]
    
    return {
        "query": query,
        "filters": filters or {},
        "results": results,
        "total_matches": len(results)
    }

# Hybrid search combining vector and keyword
@trace_retrieval(
    retrieval_type="hybrid_search",
    index_name="hybrid_index",
    capture_query=True,
    capture_results=True,
    capture_scores=True
)
def hybrid_search(query: str, alpha: float = 0.5) -> Dict[str, Any]:
    """Perform hybrid search combining vector and keyword search."""
    print(f"🔗 Hybrid Search: '{query}' with alpha={alpha}")
    
    time.sleep(0.4)
    
    # Simulate hybrid search by combining both approaches
    vector_results = vector_search(query, top_k=3)
    keyword_results = keyword_search(query)
    
    # Mock hybrid ranking
    hybrid_results = []
    for i, result in enumerate(vector_results["results"][:2]):
        hybrid_results.append({
            "document_id": result["document_id"],
            "content": result["content"],
            "vector_score": result["score"],
            "keyword_score": 0.8 - (i * 0.1),
            "combined_score": (result["score"] * alpha) + ((0.8 - i * 0.1) * (1 - alpha)),
            "source": "hybrid"
        })
    
    return {
        "query": query,
        "alpha": alpha,
        "results": hybrid_results,
        "total_results": len(hybrid_results),
        "search_strategy": "vector + keyword fusion"
    }

# Test all retrieval operations
print("🔍 Testing Retrieval System Tracing...")

# Test vector search
vector_result = vector_search("benefits of AI observability", top_k=3)
print(f"\n🔍 Vector Search Results: {len(vector_result['results'])} documents")

# Test keyword search with filters
keyword_result = keyword_search("AI monitoring", filters={"category": "technical", "year": 2024})
print(f"\n🔎 Keyword Search Results: {keyword_result['total_matches']} matches")

# Test hybrid search
hybrid_result = hybrid_search("observability tracing systems", alpha=0.7)
print(f"\n🔗 Hybrid Search Results: {len(hybrid_result['results'])} combined results")

print("\n✅ Retrieval tracing completed!")


In [ ]:
# 🔄 FLUSH AFTER CORRECTED PROXY OBJECT EXAMPLES
# This ensures all corrected proxy object traces are sent immediately

flush_traces("Corrected Proxy Objects (create_traced_agent + create_traced_openai_client)")


In [ ]:
# Enhanced Multi-Agent System Examples
# Import missing typing if not already available
from typing import Dict, Any, List

# Specialized agents with different roles and capabilities
@trace_agent(
    agent_id="data_analyst",
    role="analyst",
    agent_type="specialist",
    capabilities=["data_analysis", "statistical_modeling", "visualization"],
    capture_reasoning=True,
    metadata={"specialization": "quantitative_analysis", "confidence_threshold": 0.8}
)
def data_analyst_agent(data: Dict[str, Any]) -> Dict[str, Any]:
    """Specialized data analysis agent."""
    print(f"📊 Data Analyst: Analyzing dataset with {len(data.get('samples', []))} samples")
    
    time.sleep(0.5)
    
    # Mock data analysis
    analysis = {
        "agent_id": "data_analyst",
        "analysis_type": "quantitative",
        "findings": {
            "data_quality": 0.92,
            "pattern_confidence": 0.87,
            "anomalies_detected": 3,
            "recommendations": [
                "Data quality is high with 92% confidence",
                "3 anomalies detected requiring investigation",
                "Statistical patterns show strong correlation"
            ]
        },
        "reasoning_steps": [
            "Loaded and validated input data",
            "Applied statistical analysis methods",
            "Identified patterns and anomalies",
            "Generated confidence-based recommendations"
        ],
        "processing_time": 0.5
    }
    
    print(f"✅ Analysis complete: {analysis['findings']['data_quality']:.2f} quality score")
    return analysis

@trace_agent(
    agent_id="content_curator",
    role="curator",
    agent_type="content_specialist",
    capabilities=["content_filtering", "quality_assessment", "summarization"],
    capture_tools=True
)
def content_curator_agent(content_list: List[Dict]) -> Dict[str, Any]:
    """Content curation and quality assessment agent."""
    print(f"📝 Content Curator: Processing {len(content_list)} content items")
    
    time.sleep(0.4)
    
    # Mock content curation using tools
    high_quality_content = []
    for i, content in enumerate(content_list):
        if i < 3:  # Mock: keep first 3 as high quality
            high_quality_content.append({
                **content,
                "quality_score": 0.9 - (i * 0.05),
                "curation_reason": "Meets quality standards"
            })
    
    curation_result = {
        "agent_id": "content_curator",
        "input_count": len(content_list),
        "curated_count": len(high_quality_content),
        "curated_content": high_quality_content,
        "tools_used": ["quality_scorer", "content_filter", "summarizer"],
        "curation_metrics": {
            "retention_rate": len(high_quality_content) / len(content_list),
            "average_quality": sum(item["quality_score"] for item in high_quality_content) / len(high_quality_content)
        }
    }
    
    print(f"✅ Curated {len(high_quality_content)}/{len(content_list)} items")
    return curation_result

@trace_agent(
    agent_id="synthesis_agent",
    role="synthesizer",
    agent_type="integration_specialist", 
    capabilities=["multi_source_synthesis", "insight_generation", "report_creation"],
    capture_inputs=True,
    capture_outputs=True
)
def synthesis_agent(analyst_data: Dict, curator_data: Dict, context: str) -> Dict[str, Any]:
    """Agent that synthesizes insights from multiple sources."""
    print(f"🔗 Synthesis Agent: Combining insights for context '{context}'")
    
    time.sleep(0.6)
    
    # Synthesize insights from multiple agents
    synthesis = {
        "agent_id": "synthesis_agent",
        "context": context,
        "input_sources": ["data_analyst", "content_curator"],
        "synthesis_insights": [
            f"Data quality score of {analyst_data['findings']['data_quality']:.2f} indicates reliable foundation",
            f"Content curation retained {curator_data['curated_count']}/{curator_data['input_count']} high-quality items",
            "Cross-analysis reveals consistent quality patterns across data and content",
            "Synthesis confidence: High based on convergent evidence"
        ],
        "combined_metrics": {
            "data_quality": analyst_data['findings']['data_quality'],
            "content_quality": curator_data['curation_metrics']['average_quality'],
            "overall_confidence": (analyst_data['findings']['pattern_confidence'] + 
                                 curator_data['curation_metrics']['average_quality']) / 2
        },
        "final_recommendation": "Proceed with high confidence based on quality convergence"
    }
    
    print(f"✅ Synthesis complete with {synthesis['combined_metrics']['overall_confidence']:.2f} confidence")
    return synthesis

# Advanced orchestrator with dependency management
@trace_agent(
    agent_id="advanced_orchestrator",
    role="coordinator", 
    agent_type="orchestrator",
    capabilities=["workflow_management", "dependency_resolution", "result_aggregation"],
    capture_reasoning=True,
    metadata={"orchestration_strategy": "parallel_with_dependencies"}
)
def advanced_orchestrator(task: str, data_context: Dict) -> Dict[str, Any]:
    """Advanced orchestrator managing complex multi-agent workflows."""
    print(f"🎭 Advanced Orchestrator: Managing workflow for '{task}'")
    
    # Phase 1: Parallel execution of independent agents
    print("\n🔄 Phase 1: Parallel Agent Execution")
    
    # Mock data for demonstration
    sample_data = {"samples": [f"sample_{i}" for i in range(100)]}
    sample_content = [
        {"id": 1, "title": "AI Observability", "content": "Content about observability"},
        {"id": 2, "title": "Tracing Systems", "content": "Content about tracing"},
        {"id": 3, "title": "Monitoring Tools", "content": "Content about monitoring"},
        {"id": 4, "title": "Low Quality", "content": "Poor content"}
    ]
    
    # Execute agents in parallel (simulated)
    analyst_result = data_analyst_agent(sample_data)
    curator_result = content_curator_agent(sample_content)
    
    # Phase 2: Synthesis based on results
    print("\\n🔗 Phase 2: Synthesis and Integration")
    synthesis_result = synthesis_agent(analyst_result, curator_result, task)
    
    # Final orchestration result
    orchestration_result = {
        "task": task,
        "orchestration_id": "adv_orch_001",
        "phases": {
            "analysis": analyst_result,
            "curation": curator_result, 
            "synthesis": synthesis_result
        },
        "workflow_metrics": {
            "total_agents": 3,
            "execution_phases": 2,
            "final_confidence": synthesis_result["combined_metrics"]["overall_confidence"],
            "workflow_success": True
        },
        "reasoning": [
            "Initiated parallel execution of specialist agents",
            "Data analyst provided quantitative insights",
            "Content curator filtered and assessed quality",
            "Synthesis agent combined multi-source insights",
            "Workflow completed with high confidence"
        ]
    }
    
    print(f"\\n✅ Advanced orchestration complete!")
    return orchestration_result

# Test the enhanced multi-agent system
print("🤖 Testing Enhanced Multi-Agent System...")

# Run the advanced workflow
task = "Comprehensive analysis of AI system observability data and content"
context_data = {"domain": "ai_observability", "priority": "high"}

workflow_result = advanced_orchestrator(task, context_data)

print("\\n🎭 Enhanced Multi-Agent Results:")
print(f"Task: {workflow_result['task']}")
print(f"Agents: {workflow_result['workflow_metrics']['total_agents']}")
print(f"Phases: {workflow_result['workflow_metrics']['execution_phases']}")
print(f"Final Confidence: {workflow_result['workflow_metrics']['final_confidence']:.2f}")
print(f"Success: {workflow_result['workflow_metrics']['workflow_success']}")

print("\\n✅ Enhanced multi-agent system testing completed!")


In [ ]:
# 🔄 FLUSH AFTER ENHANCED SDK INITIALIZATION AND ENDPOINT TESTING
# This ensures the endpoint connectivity test traces are sent immediately

flush_traces("Enhanced SDK Initialization and Endpoint Testing")


In [ ]:
# Import context managers and streaming features
from noveum_trace import (
    trace_llm_call, trace_agent_operation, trace_operation, 
    streaming_llm, trace_streaming, ThreadContext
)

# Context Manager Examples - Inline Tracing

def process_user_query_with_context_managers(user_input: str) -> str:
    """Demonstrate inline tracing with context managers."""
    print(f"🔄 Processing user query: '{user_input[:40]}...'")
    
    # Some preprocessing (not traced)
    cleaned_input = user_input.strip().lower()
    
    # Trace just the LLM call using context manager
    with trace_llm_call(model="gpt-4", provider="openai", operation="query_processing") as span:
        print("🤖 Making LLM call within context manager...")
        time.sleep(0.4)
        
        # Mock LLM response
        response = f"Processed response for: {cleaned_input[:30]}..."
        
        # Add custom attributes to the span
        span.set_attributes({
            "llm.input_length": len(cleaned_input),
            "llm.output_length": len(response),
            "llm.processing_type": "query_understanding"
        })
        
        print(f"✅ LLM response generated: {len(response)} characters")
    
    # Post-processing (not traced)
    final_response = f"Final: {response}"
    print(f"📤 Final response: {final_response[:50]}...")
    
    return final_response

# Agent operation context manager
def agent_task_with_context_manager(task: str) -> Dict[str, Any]:
    """Demonstrate agent operation tracing with context manager."""
    print(f"🤖 Agent Task: '{task}'")
    
    with trace_agent_operation(
        agent_type="task_agent", 
        operation="task_execution",
        capabilities=["task_planning", "execution", "monitoring"]
    ) as span:
        print("⚙️  Executing agent task...")
        time.sleep(0.3)
        
        # Mock agent work
        result = {
            "task": task,
            "status": "completed",
            "steps_executed": 5,
            "success_rate": 0.95
        }
        
        # Add agent-specific attributes
        span.set_attributes({
            "agent.task_complexity": "medium",
            "agent.steps_executed": result["steps_executed"],
            "agent.success_rate": result["success_rate"]
        })
        
        print(f"✅ Agent task completed with {result['success_rate']:.1%} success rate")
    
    return result

# Generic operation context manager
def complex_operation_with_tracing() -> Dict[str, Any]:
    """Demonstrate generic operation tracing."""
    print("🔧 Starting complex operation...")
    
    with trace_operation(operation_name="complex_data_processing") as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "complex_data_processing",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

# Streaming LLM Examples

class MockStreamChunk:
    """Mock streaming response chunk."""
    def __init__(self, content: str):
        self.choices = [MockChoice(content)]

class MockChoice:
    """Mock choice in streaming response."""
    def __init__(self, content: str):
        self.delta = MockDelta(content)

class MockDelta:
    """Mock delta content."""
    def __init__(self, content: str):
        self.content = content

def mock_streaming_response(prompt: str) -> Iterator[MockStreamChunk]:
    """Generate mock streaming response."""
    words = f"This is a streaming response to: {prompt}. Each word comes separately.".split()
    for word in words:
        time.sleep(0.05)  # Simulate streaming delay
        yield MockStreamChunk(word + " ")

def test_streaming_with_context_manager(prompt: str) -> str:
    """Test streaming LLM with context manager."""
    print(f"🌊 Streaming LLM call: '{prompt[:30]}...'")
    
    # Create mock stream
    stream = mock_streaming_response(prompt)
    
    # Use streaming context manager
    with streaming_llm(model="gpt-4", provider="openai", operation="streaming_chat") as stream_manager:
        print("📺 Streaming response: ", end="")
        full_response = ""
        
        for chunk in stream:
            token = chunk.choices[0].delta.content
            if token:
                # Add token to stream manager for tracing
                stream_manager.add_token(token)
                print(token, end="")
                full_response += token
        
        # Add final metadata
        stream_manager.add_metadata({
            "streaming.final_length": len(full_response),
            "streaming.total_chunks": len(full_response.split())
        })
        
        print(f"\\n✅ Streaming completed: {len(full_response)} characters")
    
    return full_response.strip()

# Thread Context for Conversation Tracking

def test_thread_context_conversation() -> None:
    """Test conversation thread tracking."""
    print("💬 Testing Thread Context for Conversations...")
    
    with ThreadContext(name="demo_conversation", metadata={"session": "demo"}) as thread:
        # Simulate conversation turns
        
        # Turn 1
        thread.add_message("user", "Hello, can you help me with AI observability?")
        print("👤 User: Hello, can you help me with AI observability?")
        
        # Simulate LLM response within thread
        with trace_llm_call(model="gpt-4") as llm_span:
            time.sleep(0.3)
            response1 = "I'd be happy to help you with AI observability!"
            thread.add_message("assistant", response1)
            print(f"🤖 Assistant: {response1}")
        
        # Turn 2  
        thread.add_message("user", "What are the key components?")
        print("👤 User: What are the key components?")
        
        with trace_llm_call(model="gpt-4") as llm_span:
            time.sleep(0.4)
            response2 = "Key components include tracing, metrics, and logging."
            thread.add_message("assistant", response2)
            print(f"🤖 Assistant: {response2}")
        
        # Get thread statistics
        stats = thread.get_statistics()
        print(f"\\n📊 Thread Stats: {stats['message_count']} messages, {stats['turn_count']} turns")

# Test all context manager and streaming features
print("🔄 Testing Context Managers and Streaming...")

# Test context managers
print("\\n1️⃣ Context Manager Examples:")
query_result = process_user_query_with_context_managers("What are the benefits of AI observability?")

agent_result = agent_task_with_context_manager("Analyze system performance metrics")

operation_result = complex_operation_with_tracing()

# Test streaming
print("\\n2️⃣ Streaming Examples:")
stream_result = test_streaming_with_context_manager("Explain machine learning concepts")

# Test thread context
print("\\n3️⃣ Thread Context Examples:")
test_thread_context_conversation()

print("\\n✅ Context managers and streaming testing completed!")


In [ ]:
# CORRECTED trace_operation Examples
# The correct syntax is: trace_operation(operation_name, attributes=dict, tags=dict)

def test_correct_trace_operation_usage():
    """Demonstrate the CORRECT syntax for trace_operation context manager."""
    print("🔧 Testing CORRECT trace_operation usage...")
    
    # ✅ CORRECT: First parameter is operation_name (string), second is attributes (dict)
    with trace_operation("data_processing_pipeline", 
                        attributes={"operation_type": "data_pipeline", "complexity": "high"}) as span:
        # Step 1: Data loading
        print("📥 Step 1: Loading data...")
        time.sleep(0.2)
        span.set_attributes({"step": "data_loading", "records_loaded": 1000})
        
        # Step 2: Processing
        print("⚙️  Step 2: Processing data...")
        time.sleep(0.3)
        span.set_attributes({"step": "processing", "records_processed": 950})
        
        # Step 3: Output
        print("📤 Step 3: Generating output...")
        time.sleep(0.1)
        span.set_attributes({"step": "output", "records_output": 950})
        
        result = {
            "operation": "data_processing_pipeline",
            "input_records": 1000,
            "processed_records": 950,
            "success": True
        }
        
        print("✅ Complex operation completed successfully")
    
    return result

def test_correct_batch_operations():
    """Demonstrate correct trace_operation usage in batch processing."""
    print("📦 Testing CORRECT batch trace_operation usage...")
    
    operations = []
    
    for i in range(3):  # Reduced to 3 for demo
        # ✅ CORRECT: operation_name first, then attributes dict
        with trace_operation(f"batch_operation_{i}", 
                           attributes={"operation_type": "batch_demo", "batch_index": i}) as span:
            span.set_attributes({
                "batch.operation_number": i,
                "batch.total_operations": 3,
                "operation.size": "small"
            })
            time.sleep(0.1)  # Quick operations
            operations.append(f"operation_{i}")
            print(f"🔸 Batch operation {i+1}/3 completed")
    
    print(f"✅ Batch processing completed: {len(operations)} operations")
    return operations

# Test the corrected functions
print("🔧 Testing CORRECTED Context Manager Usage...")

print("\\n1️⃣ Correct trace_operation Usage:")
operation_result = test_correct_trace_operation_usage()

print("\\n2️⃣ Correct Batch Operations:")
batch_result = test_correct_batch_operations()

print(f"\\n✅ All corrected context manager examples completed!")
print(f"Operation result: {operation_result['success']}")
print(f"Batch operations: {len(batch_result)} completed")


In [ ]:
# CORRECTED create_traced_agent Example
# The correct signature is: create_traced_agent(agent, agent_type, capabilities, trace_config)

def test_traced_agent_proxy_corrected():
    """Test traced agent proxy with CORRECT parameters."""
    print("🤖 Testing Traced Agent Proxy (CORRECTED)...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # ✅ CORRECT: Use proper parameter names and structure
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        agent_type="traced_demo_agent",  # ✅ CORRECT: agent_type (not agent_id)
        capabilities=["thinking", "acting", "planning"],  # ✅ CORRECT: capabilities (not auto_trace_methods)
        trace_config={"capture_inputs": True, "capture_outputs": True}  # ✅ CORRECT: trace_config dict
    )
    
    print("✅ Traced agent proxy created successfully!")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")
    
    return traced_agent

def test_traced_openai_client_corrected():
    """Test traced OpenAI client with CORRECT parameters."""
    print("🔄 Testing Traced OpenAI Client (CORRECTED)...")
    
    # ✅ CORRECT: Use actual OpenAI client instance, not direct parameters
    try:
        # First create a real OpenAI client (even with mock key)
        import openai
        original_client = openai.OpenAI(api_key="mock-key-for-demo")
        
        # ✅ CORRECT: Pass the client instance to the tracer
        traced_client = create_traced_openai_client(
            original_client=original_client,
            trace_config={"trace_completions": True, "capture_content": True}
        )
        print("✅ Traced OpenAI client created successfully!")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")
        print("📝 Note: This requires a real OpenAI client instance")

# Test the corrected functions
print("🚀 Testing CORRECTED Proxy Object Functions...")

print("\\n1️⃣ Corrected Traced Agent:")
traced_agent = test_traced_agent_proxy_corrected()

print("\\n2️⃣ Corrected Traced OpenAI Client:")
test_traced_openai_client_corrected()

print("\\n✅ All corrected proxy object examples completed!")


In [ ]:
# 🔧 ENHANCED SDK INITIALIZATION WITH ENDPOINT DEBUGGING
import noveum_trace
from noveum_trace import trace, trace_agent, trace_llm, trace_tool
import logging
import sys
import requests

# 🔍 COMPREHENSIVE DEBUGGING SETUP
print("🔧 Setting up enhanced debugging for transport layer...")

# Set up comprehensive logging with detailed output
logging.basicConfig(
    level=logging.DEBUG, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Enable specific loggers for transport debugging
loggers_to_enable = [
    'noveum_trace.transport', 
    'noveum_trace.transport.http_transport',
    'urllib3.connectionpool'
]

for logger_name in loggers_to_enable:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)

print("✅ Enhanced debugging enabled")

# 🌐 ENDPOINT VERIFICATION 
def verify_endpoint():
    \"\"\"Verify that the Beeceptor endpoint is reachable.\"\"\"
    endpoint = "https://noveum-trace.free.beeceptor.com"
    
    print(f"🔍 Verifying endpoint reachability: {endpoint}")
    
    try:
        response = requests.get(endpoint, timeout=10)
        print(f"✅ Base endpoint reachable - Status: {response.status_code}")
        
        # Test the actual trace endpoints
        trace_endpoints = [
            f"{endpoint}/v1/trace",   # Single trace endpoint
            f"{endpoint}/v1/traces"   # Batch trace endpoint  
        ]
        
        for test_endpoint in trace_endpoints:
            try:
                test_response = requests.head(test_endpoint, timeout=5)
                print(f"📡 {test_endpoint} - Status: {test_response.status_code}")
            except Exception as e:
                print(f"⚠️  {test_endpoint} - Error: {e}")
                
    except Exception as e:
        print(f"❌ Endpoint verification failed: {e}")
        return False
    
    return True

# Verify endpoint first
endpoint_ok = verify_endpoint()


In [ ]:
# 🚀 ENHANCED SDK INITIALIZATION
try:
    print("\\n🚀 Initializing Noveum Trace SDK with enhanced configuration...")
    
    # 📋 Display endpoint mapping
    base_endpoint = "https://api.noveum.ai/api"
    print(f"🌐 Base Endpoint: {base_endpoint}")
    print(f"📡 Single Trace Endpoint: {base_endpoint}/v1/trace")
    print(f"📦 Batch Trace Endpoint: {base_endpoint}/v1/traces")
    print(f"🔧 Transport Mode: Individual traces (batch_size=1) for better debugging")
    
    noveum_trace.init(
        api_key=os.getenv('NOVEUM_API_KEY'),
        project="jupyter-test-project",
        environment="development", 
        endpoint=base_endpoint,  # SDK will append /v1/trace or /v1/traces automatically
        debug=True,  # Enable debug mode
        
        # 🔧 Enhanced transport configuration for debugging
        transport_config={
            "timeout": 30,           # 30 second timeout (generous for debugging)
            "retry_attempts": 0,     # No retries for faster debugging feedback  
            "batch_size": 1,         # Send traces individually (not batched)
            "batch_timeout": 0.5,    # Send traces immediately
            "compression": False,    # No compression for easier debugging
            "verify_ssl": True       # Verify SSL certificates
        },
        
        # ✅ Comprehensive tracing configuration  
        tracing_config={
            "sample_rate": 1.0,        # Trace 100% of operations
            "capture_errors": True,    # Capture error details
            "auto_flush": True         # Automatically flush traces
        }
    )
    
    print("✅ Noveum Trace SDK initialized successfully!")
    
    # 📊 Display configuration details
    config = noveum_trace.get_config()
    print(f"📊 Project: {config.project}")
    print(f"🔧 Environment: {config.environment}")
    print(f"🌐 Transport Endpoint: {config.transport.endpoint}")
    print(f"📦 Batch Size: {config.transport.batch_size}")
    print(f"⏱️  Batch Timeout: {config.transport.batch_timeout}s")
    print(f"🔍 Debug Mode: {config.debug}")
    
    print("\\n🎯 SDK initialization completed!")
    print("📋 Check the debug log output above for HTTP request details")
    print("🌐 Your traces should now be visible at: " + base_endpoint)
    
except Exception as e:
    print(f"❌ Error initializing SDK: {e}")
    print("Continuing with demo - traces will be logged locally")
    import traceback
    traceback.print_exc()


In [ ]:
# Import advanced features
from noveum_trace import (
    auto_instrument, get_instrumented_libraries, is_instrumented,
    create_traced_openai_client, create_traced_agent, TracedOpenAIClient,
    start_trace, start_span, get_current_trace, get_current_span
)

# Auto-Instrumentation Examples

def test_auto_instrumentation():
    """Test automatic instrumentation of libraries."""
    print("🔧 Testing Auto-Instrumentation...")
    
    # Check available instrumentations
    available = noveum_trace.get_available_instrumentations()
    print(f"📦 Available instrumentations: {available}")
    
    # Enable auto-instrumentation for OpenAI (if not already enabled)
    if not is_instrumented("openai"):
        print("🔌 Enabling OpenAI auto-instrumentation...")
        try:
            auto_instrument("openai")
            print("✅ OpenAI auto-instrumentation enabled")
        except Exception as e:
            print(f"⚠️  Auto-instrumentation: {e}")
    else:
        print("✅ OpenAI already instrumented")
    
    # Check instrumented libraries
    instrumented = get_instrumented_libraries()
    print(f"🔍 Currently instrumented: {instrumented}")
    
    return instrumented

# Proxy Objects for Enhanced Control

def test_traced_openai_client():
    """Test traced OpenAI client proxy."""
    print("🔄 Testing Traced OpenAI Client...")
    
    # Create traced OpenAI client (even without real API key)
    try:
        traced_client = create_traced_openai_client(
            api_key="mock-key-for-demo",
            trace_completions=True,
            trace_embeddings=True,
            capture_content=True
        )
        print("✅ Traced OpenAI client created")
        
        # Mock a call (won't actually work without real API key)
        print("🤖 Simulating traced OpenAI call...")
        # In real usage: response = traced_client.chat.completions.create(...)
        print("ℹ️  Would automatically trace all OpenAI API calls")
        
    except Exception as e:
        print(f"ℹ️  Traced client demo: {e}")

def test_traced_agent_proxy():
    """Test traced agent proxy for enhanced agent monitoring."""
    print("🤖 Testing Traced Agent Proxy...")
    
    # Mock agent class
    class MockAgent:
        def __init__(self, name: str):
            self.name = name
        
        def think(self, problem: str) -> str:
            time.sleep(0.2)
            return f"Thinking about: {problem}"
        
        def act(self, action: str) -> str:
            time.sleep(0.3)
            return f"Performing action: {action}"
        
        def plan(self, goal: str) -> List[str]:
            time.sleep(0.4)
            return [f"Step 1 for {goal}", f"Step 2 for {goal}", f"Step 3 for {goal}"]
    
    # Create traced agent proxy
    original_agent = MockAgent("demo_agent")
    traced_agent = create_traced_agent(
        agent=original_agent,
        auto_trace_methods=["think", "act", "plan"],
        capture_inputs=True,
        capture_outputs=True
    )
    
    print("✅ Traced agent proxy created")
    
    # Test traced methods
    print("🧠 Testing traced agent methods...")
    
    thought = traced_agent.think("How to improve AI observability")
    print(f"💭 Think result: {thought}")
    
    action = traced_agent.act("Implement monitoring dashboard")
    print(f"⚡ Action result: {action}")
    
    plan = traced_agent.plan("Enhance system reliability")
    print(f"📋 Plan result: {len(plan)} steps")

# Manual Span Creation and Management

def test_manual_tracing():
    """Test manual trace and span creation."""
    print("🔍 Testing Manual Tracing...")
    
    # Start a manual trace
    trace = start_trace("manual_demo_trace")
    print(f"✅ Started trace: {trace.trace_id}")
    
    # Create nested spans manually
    with trace.span("parent_operation") as parent_span:
        parent_span.set_attributes({
            "operation.type": "parent",
            "operation.importance": "high"
        })
        print("📊 Parent span created")
        
        # Child span 1
        with parent_span.create_child_span("child_operation_1") as child1:
            child1.set_attributes({
                "operation.type": "child",
                "child.number": 1
            })
            time.sleep(0.2)
            print("🔹 Child span 1 completed")
        
        # Child span 2  
        with parent_span.create_child_span("child_operation_2") as child2:
            child2.set_attributes({
                "operation.type": "child",
                "child.number": 2,
                "child.data_processed": 500
            })
            time.sleep(0.3)
            print("🔹 Child span 2 completed")
        
        print("📊 Parent operation completed")
    
    # Finish trace
    trace.finish()
    print(f"✅ Manual trace completed: {trace.trace_id}")

# Advanced Configuration and Performance Features

def test_advanced_configuration():
    """Test advanced SDK configuration options."""
    print("⚙️  Testing Advanced Configuration...")
    
    # Get current configuration
    config = noveum_trace.get_config()
    print(f"📋 Current project: {config.project}")
    print(f"🌐 Current endpoint: {config.transport.endpoint}")
    print(f"📦 Batch size: {config.transport.batch_size}")
    print(f"⏱️  Batch timeout: {config.transport.batch_timeout}")
    
    # Test configuration updates (temporary for demo)
    original_debug = config.debug
    
    # Temporarily enable debug mode
    noveum_trace.configure(debug=True)
    print("🐛 Debug mode enabled temporarily")
    
    # Create a trace to demonstrate debug output
    with noveum_trace.trace_operation("debug_demo_operation") as span:
        span.set_attributes({"demo": "configuration", "debug_enabled": True})
        time.sleep(0.1)
        print("✅ Debug operation completed")
    
    # Restore original debug setting
    noveum_trace.configure(debug=original_debug)
    print(f"🔧 Debug mode restored to: {original_debug}")

# Batch Processing and Performance Monitoring

def test_batch_processing():
    """Test batch operations for performance."""
    print("📦 Testing Batch Processing...")
    
    # Create multiple operations quickly to test batching
    operations = []
    
    for i in range(5):
        with noveum_trace.trace_operation(f"batch_operation_{i}", operation_type="batch_demo") as span:
            span.set_attributes({
                "batch.operation_number": i,
                "batch.total_operations": 5,
                "operation.size": "small"
            })
            time.sleep(0.05)  # Quick operations
            operations.append(f"operation_{i}")
            print(f"🔸 Batch operation {i+1}/5 completed")
    
    print(f"✅ Batch processing completed: {len(operations)} operations")
    
    # Force flush to send batched traces
    try:
        noveum_trace.flush()
        print("📤 Forced flush of batched traces")
    except Exception as e:
        print(f"ℹ️  Flush status: {e}")

# Error Handling and Edge Cases

def test_error_handling():
    """Test error handling and edge cases."""
    print("⚠️  Testing Error Handling...")
    
    # Test error capture in traced function
    @noveum_trace.trace(capture_errors=True, capture_stack_trace=True)
    def operation_with_error(should_fail: bool = False):
        if should_fail:
            raise ValueError("This is a demo error for testing")
        return "Success!"
    
    # Test successful operation
    try:
        result = operation_with_error(should_fail=False)
        print(f"✅ Successful operation: {result}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    
    # Test error capture
    try:
        result = operation_with_error(should_fail=True)
        print(f"Unexpected success: {result}")
    except ValueError as e:
        print(f"✅ Error captured successfully: {e}")

# Run all advanced feature tests
print("🚀 Testing Auto-Instrumentation and Advanced Features...")

print("\\n1️⃣ Auto-Instrumentation:")
instrumented_libs = test_auto_instrumentation()

print("\\n2️⃣ Proxy Objects:")
test_traced_openai_client()
test_traced_agent_proxy()

print("\\n3️⃣ Manual Tracing:")
test_manual_tracing()

print("\\n4️⃣ Advanced Configuration:")
test_advanced_configuration()

print("\\n5️⃣ Batch Processing:")
test_batch_processing()

print("\\n6️⃣ Error Handling:")
test_error_handling()

print("\\n✅ Advanced features testing completed!")


In [ ]:
@trace_agent(agent_id="research_agent")
def research_agent(query: str) -> Dict[str, Any]:
    """Research agent that gathers information."""
    print(f"🔍 Research Agent: Processing query '{query}'")

    # Simulate research process
    time.sleep(0.4)

    # Mock research findings
    findings = {
        "query": query,
        "sources": ["source1.pdf", "source2.html", "source3.json"],
        "key_points": [
            "Point 1: Observability improves system reliability",
            "Point 2: Tracing helps identify bottlenecks",
            "Point 3: Monitoring enables proactive maintenance"
        ],
        "confidence": 0.87,
        "research_time": "0.4s"
    }

    print(f"✅ Research completed with {len(findings['sources'])} sources")
    return findings

@trace_agent(agent_id="orchestrator")
def orchestrate_workflow(task: str) -> Dict[str, Any]:
    """Orchestrator agent that coordinates multiple agents."""
    print(f"🎭 Orchestrator: Starting workflow for task '{task}'")

    # Step 1: Research
    print("\n🔍 Step 1: Research Phase")
    research_data = research_agent(task)

    # Step 2: Analysis (simplified)
    print("\n📊 Step 2: Analysis Phase")
    analysis_data = {
        "insights": ["Observability is crucial", "Tracing provides insights"],
        "quality_score": 0.89
    }

    # Final orchestration result
    workflow_result = {
        "task": task,
        "workflow_id": "wf-001",
        "phases_completed": 2,
        "research_summary": research_data["key_points"],
        "analysis_summary": analysis_data["insights"],
        "overall_confidence": research_data["confidence"],
        "total_time": "1.0s"
    }

    print("\n✅ Orchestrator: Workflow completed successfully")
    return workflow_result

# Test the multi-agent workflow
task = "Analyze the importance of observability in AI systems"
workflow_result = orchestrate_workflow(task)
print("\n🎭 Final Workflow Result:")
print(f"Task: {workflow_result['task']}")
print(f"Confidence: {workflow_result['overall_confidence']:.2f}")
print(f"Phases: {workflow_result['phases_completed']}")


In [ ]:
@trace_tool(tool_name="calculator")
def calculate(operation: str, a: float, b: float) -> Dict[str, Any]:
    """A calculator tool with tracing."""
    print(f"🔢 Calculator: Performing {operation} on {a} and {b}")

    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else None
    }

    if operation not in operations:
        return {"error": f"Unknown operation: {operation}"}

    try:
        result = operations[operation](a, b)
        if result is None:
            return {"error": "Division by zero"}

        return {
            "operation": operation,
            "operands": [a, b],
            "result": result,
            "success": True
        }
    except Exception as e:
        return {"error": str(e), "success": False}

@trace_tool(tool_name="text_analyzer")
def analyze_text(text: str) -> Dict[str, Any]:
    """Text analysis tool with tracing."""
    print(f"📝 Text Analyzer: Analyzing text of length {len(text)}")

    # Simulate analysis
    time.sleep(0.2)

    analysis = {
        "text_length": len(text),
        "word_count": len(text.split()),
        "sentence_count": text.count('.') + text.count('!') + text.count('?'),
        "avg_word_length": sum(len(word) for word in text.split()) / len(text.split()) if text.split() else 0
    }

    print(f"✅ Analysis complete: {analysis['word_count']} words, {analysis['sentence_count']} sentences")
    return analysis

# Test tool tracing
calc_result = calculate("multiply", 15, 4)
print(f"\n🔢 Calculator Result: {calc_result}")

text_to_analyze = "This is a sample text for testing the noveum-trace SDK. It contains multiple sentences!"
text_analysis = analyze_text(text_to_analyze)
print(f"\n📝 Text Analysis Result: {text_analysis}")


In [ ]:
import sys
import pkg_resources

# Test summary
def print_test_summary():
    """Print a summary of all tests performed."""
    print("📋 NOVEUM TRACE SDK TEST SUMMARY")
    print("=" * 50)

    # Check SDK version
    try:
        version = pkg_resources.get_distribution("noveum-trace").version
        print(f"✅ SDK Version: {version}")
    except:
        print("⚠️  Could not determine SDK version")

    # Environment check
    print(f"✅ Python Version: {sys.version.split()[0]}")
    print(f"✅ Environment Variables: {'✓' if os.getenv('NOVEUM_API_KEY') else '✗'}")

    # Features tested
    features_tested = [
        "Basic function tracing (@trace)",
        "LLM call tracing (@trace_llm)",
        "Agent workflow tracing (@trace_agent)",
        "Tool tracing (@trace_tool)",
        "Multi-agent orchestration",
        "Error handling",
        "Framework integration simulation"
    ]

    print("\n🧪 Features Tested:")
    for feature in features_tested:
        print(f"  ✅ {feature}")

    print("\n🎯 Key Results:")
    print("  🔧 All decorators: Functional")
    print("  🤖 Multi-agent support: Functional")
    print("  🔌 Framework integration: Simulated successfully")

    print("\n✅ All tests completed successfully!")
    print("\n📖 Next Steps:")
    print("  1. Set up your actual NOVEUM_API_KEY for production use")
    print("  2. Integrate with your LLM applications")
    print("  3. Set up dashboards and monitoring")
    print("  4. Configure alerting based on trace data")

# Clean up function
def cleanup_resources():
    """Clean up any resources created during testing."""
    print("🧹 Cleaning up test resources...")

    try:
        # Attempt to flush any pending traces
        noveum_trace.flush()
        print("✅ Traces flushed successfully")
    except Exception as e:
        print(f"ℹ️  Trace flush: {e}")

    print("✅ Cleanup completed")

# Run summary and cleanup
print_test_summary()
cleanup_resources()
